# Yolov8 Model Training

This notebook helps easily train a yolo model. Model evaluation will happen in another notebook.

## 1. Imports and variable setup

In [1]:
# Import all libraries
import os
from roboflow import Roboflow
from IPython import display
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Setup HOME environment variable
HOME = os.getcwd()
print(HOME)

/home/mreag/repos/DBD-Killer-AI/notebooks


In [2]:
# Check image displays are good
# display.clear_output()
sys.path.append(str(Path.cwd().parent))
# Check ultralytics library is good
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Setup complete ✅ (12 CPUs, 7.7 GB RAM, 70.5/1006.9 GB disk)


In [3]:
# Environment variables
PROJECT_DIR = Path.cwd().parent
DATASET_VERSION = 7

## 2. Import Model

Pre-trained is the ideal model

In [4]:
model = YOLO(model=str(PROJECT_DIR) + "/models/yolov8n.pt")

## 3. Train Model

In [5]:
str(PROJECT_DIR) + "/data/external"

'/home/mreag/repos/DBD-Killer-AI/data/external'

In [6]:
from dbdkillerai.data.make_dataset import roboflow_connect, roboflow_download
from dbdkillerai.models.train_model import train_yolo, validate_yolo

# Establish Roboflow connection and acquire dataset location. DONT download.
rf_conn, rf_project = roboflow_connect()
data_location = roboflow_download(rf_project=rf_project,
                                  rf_data_version=DATASET_VERSION,
                                  data_format="yolov8",
                                  project_dir=str(PROJECT_DIR) + "/data/external",
                                  overwrite=True)

# Get location of yml
yml_location = data_location.location + "/data.yaml"
model, results_train = train_yolo(yolo_model=model,
                                    data_yml=yml_location,
                                    epochs=150,
                                    imgsz=800,
                                    plots=True,
                                    workers=0)



loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.227, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7 in yolov8:: 100%|██████████| 3614/3614 [00:00<00:00, 5825.92it/s]


New https://pypi.org/project/ultralytics/8.1.3 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine/trainer: task=detect, mode=train, model=/home/mreag/repos/DBD-Killer-AI/models/yolov8n.pt, data=/home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7/data.yaml, epochs=150, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

train: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7/train/labels.cache... 1444 images, 454 backgrounds, 0 corrupt: 100%|██████████| 1444/1444 [00:00<?, ?it/s]
val: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7/valid/labels.cache... 238 images, 80 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]

Plotting labels to runs/detect/train19/labels.jpg... 


...

    150/150      3.28G     0.4419     0.2932     0.8088          7        800: 100%|██████████| 91/91 [00:28<00:00,  3.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.48it/s]

                   all        238        343      0.836      0.728      0.773      0.518



150 epochs completed in 1.354 hours.
Optimizer stripped from runs/detect/train19/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train19/weights/best.pt, 6.3MB

Validating runs/detect/train19/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.44it/s]


                   all        238        343      0.805      0.712      0.765      0.527
              activity        238         12      0.814       0.73      0.777      0.511
             generator        238        189      0.859      0.884      0.904      0.654
                  hook        238        120      0.851      0.733      0.791      0.532
              survivor        238         22      0.697        0.5      0.589      0.409
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train19


In [7]:
data_location

## Validate Custom Model

In [8]:
best_weights_location = str(results_train.save_dir) + "/weights/best.pt"
best_model = YOLO(best_weights_location)
results_val = validate_yolo(yolo_model=best_model,
                            data_yml=yml_location)


Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients


val: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/7/valid/labels.cache... 238 images, 80 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.26it/s]


                   all        238        343      0.806      0.713      0.765       0.52
              activity        238         12      0.815      0.733      0.777      0.501
             generator        238        189      0.862      0.884      0.904      0.656
                  hook        238        120      0.855      0.734      0.791      0.526
              survivor        238         22      0.694        0.5      0.588      0.396
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val5
